In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
import pymysql
from sqlite3 import dbapi2 as sq3
from pathlib import Path

import time
import copy
import random

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from dateutil import rrule

from io import BytesIO
from collections import OrderedDict
PATHSTART = '.'

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# function to connect to wiki_dbs and query
def connect_and_query(wiki_name, query, cnf_path = "/home/paws/.my.cnf"):
    
    conn = pymysql.connect(host = wiki_name + '.analytics.db.svc.wikimedia.cloud',
                           read_default_file = cnf_path,
                           database = wiki_name + '_p')
    
    with conn.cursor() as cur:
        cur.execute(query)
        data = cur.fetchall()
    
    conn.close()
    return data

In [8]:
def create_db(dbfile, schema):
    db = sq3.connect(Path(PATHSTART) / dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db

# schema for the database
db_schemas = """
DROP TABLE IF EXISTS "newusers";

CREATE TABLE "newusers" (
    "user_name" VARCHAR,
    "cohort" INT,
    "campaign" VARCHAR,
    "cat" VARCHAR,
    "country" VARCHAR);
    
CREATE TABLE "alledits" (
    "wiki_db" VARCHAR,
    "event_entity" VARCHAR,
    "event_timestamp" TIMESTAMP,
    "event_user_text" VARCHAR,
    "event_user_registration_timestamp" TIMESTAMP,
    "revision_id" INTEGER);
"""

In [9]:
#master_db = create_db('master.db', db_schemas)
master_db = sq3.connect('master.db')

In [10]:
wlc_df = pd.read_csv('major_campaigns_timelines.csv')
wlc_df

,year,country,start_date,end_date,cat,campaign
0,2014,with_no_country,20141001000000,20141130235959,Images_from_Wiki_Loves_Africa_2014_in_an_unkno...,Wiki Loves Africa
1,2014,Algeria,20141001000000,20141130235959,Images_from_Wiki_Loves_Africa_2014_in_Algeria,Wiki Loves Africa
2,2014,Angola,20141001000000,20141130235959,Images_from_Wiki_Loves_Africa_2014_in_Angola,Wiki Loves Africa
3,2014,Benin,20141001000000,20141130235959,Images_from_Wiki_Loves_Africa_2014_in_Benin,Wiki Loves Africa
4,2014,Botswana,20141001000000,20141130235959,Images_from_Wiki_Loves_Africa_2014_in_Botswana,Wiki Loves Africa
...,...,...,...,...,...,...
1241,2017,with_no_country,20170901000000,20170930235959,Images_from_Wiki_Loves_Monuments_2017,Wiki Loves Monuments
1242,2018,with_no_country,20180901000000,20180930235959,Images_from_Wiki_Loves_Monuments_2018,Wiki Loves Monuments
1243,2019,with_no_country,20190901000000,20190930235959,Images_from_Wiki_Loves_Monuments_2019,Wiki Loves Monuments
1244,2020,with_no_country,20200901000000,20201107235959,Images_from_Wiki_Loves_Monuments_2020,Wiki Loves Monuments


In [11]:
# code to query list of users from each campaign and store it in database
for i in wlc_df.index:  
    query1=f"""
    WITH
    cat_filter AS (
      SELECT rev_id, rev_actor
      FROM categorylinks
      INNER JOIN revision ON cl_from = rev_page
      WHERE cl_to = "{wlc_df['cat'][i]}" AND cl_type = "file" AND rev_parent_id = 0),

    newuser_filter AS (
      SELECT DISTINCT actor_user, actor_id
      FROM cat_filter
      INNER JOIN actor ON actor_id = rev_actor AND rev_actor IS NOT NULL
      INNER JOIN user ON actor_user = user_id AND user_id IS NOT NULL
      WHERE user_registration BETWEEN "{wlc_df['start_date'][i]}" and "{wlc_df['end_date'][i]}"),

    edit_filter AS (
      SELECT user_name, user_registration, user_id, rev_timestamp, rev_id
      FROM newuser_filter
      LEFT JOIN user ON actor_user = user_id
      LEFT JOIN revision ON actor_id = rev_actor AND rev_actor IS NOT NULL),

    notbot_filter AS (
      SELECT *
      FROM edit_filter
      LEFT JOIN user_groups on ug_user = user_id
      WHERE (ug_group IS NULL) OR (ug_group NOT LIKE '%bot%')
    ),
  
    month_filter AS (
        SELECT user_name, YEAR(user_registration) AS year, COUNT(rev_timestamp), user_registration
        FROM notbot_filter
        GROUP BY user_name)

    SELECT user_name, year AS cohort
    FROM month_filter
    """

    output1 = connect_and_query('commonswiki', query1)
    temp_df = pd.DataFrame(output1, columns=['user_name', 'cohort'])
    temp_df['user_name'] = temp_df['user_name'].apply(lambda x:x.decode("unicode_escape").encode('raw_unicode_escape').decode('utf-8'))
    temp_df['campaign'] = wlc_df['campaign'][i]
    temp_df['cat'] = wlc_df['cat'][i]
    temp_df['country'] = wlc_df['country'][i]
    #print(Appended info of str(temp_df.shape[0]) + ' new editors from ' + wlc_df['campaign'][i] +' '+ str(wlc_df['year'][i]) +', '+ wlc_df['country'][i])
    temp_df.to_sql("newusers", master_db, if_exists='append', index=False)
    del temp_df
    del output1
print('list of new users has been successfullly appended to the database')

6 new editors from Wiki Loves Africa 2014, with_no_country
24 new editors from Wiki Loves Africa 2014, Algeria
9 new editors from Wiki Loves Africa 2014, Angola
3 new editors from Wiki Loves Africa 2014, Benin
4 new editors from Wiki Loves Africa 2014, Botswana
2 new editors from Wiki Loves Africa 2014, Burkina_Faso
6 new editors from Wiki Loves Africa 2014, Burundi
10 new editors from Wiki Loves Africa 2014, Cameroon
2 new editors from Wiki Loves Africa 2014, Cape_Verde
0 new editors from Wiki Loves Africa 2014, Central African Republic
2 new editors from Wiki Loves Africa 2014, Chad
0 new editors from Wiki Loves Africa 2014, Comoros
6 new editors from Wiki Loves Africa 2014, Democratic Republic of the Congo
1 new editors from Wiki Loves Africa 2014, Republic of the Congo
31 new editors from Wiki Loves Africa 2014, Côte_d'Ivoire
0 new editors from Wiki Loves Africa 2014, Djibouti
34 new editors from Wiki Loves Africa 2014, Egypt
0 new editors from Wiki Loves Africa 2014, Equatorial_Gu

In [19]:
# removing duplicates from the database
db_df = pd.read_sql_query("""SELECT * FROM newusers""", master_db)
db_df = db_df.replace(to_replace='an_unknown_country', value='with_no_country').sort_values('country')
db_df.drop_duplicates(subset=['user_name'], keep='first', inplace=True)
db_df.to_sql("newusers", master_db, if_exists='replace', index=False)

146221